In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import differential_evolution
from tensorflow.keras.models import load_model

model = load_model("lstm_stress_alpha_test.keras")
sequence_length = 60

def generate_custom_sequence(sequence_length, baseline, fluctuation, noise_level, trend=0.0, frequency=1.0):
    """Generate synthetic sequence with specified parameters."""
    noise = np.random.normal(0, noise_level, sequence_length)
    time = np.arange(sequence_length)
    trend_component = trend * time / sequence_length  
    fluctuation_component = fluctuation * np.sin(2 * np.pi * frequency * time / sequence_length)
    sequence = baseline + trend_component + fluctuation_component + noise
    return np.clip(sequence, 0, 1)  


def evaluate_parameters(params, target_class_index, num_sequences=200):
    baseline, fluctuation, noise_level, trend, frequency = params
    sequences = np.array([generate_custom_sequence(sequence_length, baseline, fluctuation, noise_level, trend, frequency)
                          for _ in range(num_sequences)])
    sequences = sequences.reshape(num_sequences, sequence_length, 1)
    
    predictions = model.predict(sequences, verbose=0)
    predicted_classes = np.argmax(predictions, axis=1)
    correct_count = np.sum(predicted_classes == target_class_index)
    accuracy = correct_count / num_sequences
    return -accuracy 


stress_classes = ['High', 'Low', 'VeryHigh', 'Medium'] #Index of classes should be same as that of models
optimized_parameters = {}

for class_index, class_name in enumerate(stress_classes):
    print(f"\nOptimizing parameters for class: {class_name}")


    bounds = [(0.1, 1.0),    # baseline 
              (0.05, 1.0),   # fluctuation 
              (0.01, 0.5),   # noise_level 
              (-0.2, 0.2),   # trend 
              (0.2, 5.0)]    # frequency 


    result = differential_evolution(
        evaluate_parameters,
        bounds,
        args=(class_index,),
        maxiter=200,
        popsize=20,
        tol=0.01,
        disp=False  
    )


    optimized_parameters[class_name] = result.x
    print(f"Optimal parameters for {class_name}: {result.x}")


for class_name, params in optimized_parameters.items():
    print(f"\nClass: {class_name}")
    print(f" - Baseline: {params[0]:.2f}")
    print(f" - Fluctuation: {params[1]:.2f}")
    print(f" - Noise Level: {params[2]:.2f}")
    print(f" - Trend: {params[3]:.2f}")
    print(f" - Frequency: {params[4]:.2f}")


for class_name, params in optimized_parameters.items():
    baseline, fluctuation, noise_level, trend, frequency = params
    sample_sequence = generate_custom_sequence(sequence_length, baseline, fluctuation, noise_level, trend, frequency)
    
    plt.figure()
    plt.plot(sample_sequence, label=f"{class_name} Sample Sequence")
    plt.title(f"Optimized Sequence Pattern for {class_name}")
    plt.xlabel("Sequence Step")
    plt.ylabel("Signal Value")
    plt.legend()
    plt.show()
